In [67]:
import pandas as pd
import os
import glob

In [68]:
file_dir = r'C:/Users/shine/Desktop/BigData/Mentoring/0919'
save_file = r'C:/Users/shine/Desktop/BigData/Mentoring/0919/서울 20~30대 생활인구(6~8월).csv'
all_file = glob.glob(os.path.join(file_dir, 'LOCAL_PEOPLE_DONG_20200*.csv'))

In [69]:
people_data = []
for file in all_file:
    people = pd.read_csv(file, index_col=False)
    people = people[['기준일ID','행정동코드','시간대구분',
                     '남자20세부터24세생활인구수',
                     '남자25세부터29세생활인구수',
                     '남자30세부터34세생활인구수',
                     '남자35세부터39세생활인구수',
                     '여자20세부터24세생활인구수',
                     '여자25세부터29세생활인구수',
                     '여자30세부터34세생활인구수',
                     '여자35세부터39세생활인구수'
                    ]]
    people_data.append(people)

In [70]:
# 한 df로 묶기
dataCombine = pd.concat(people_data, axis=0, ignore_index = True)
dataCombine = dataCombine.sort_values(by=['기준일ID','행정동코드'])
dataCombine = dataCombine.round(0)
dataCombine = dataCombine.astype(int, errors = 'ignore')
dataCombine = dataCombine.astype({'행정동코드':'str'})
dataCombine['행정동코드'] = dataCombine['행정동코드'].str[0:5]
#dataCombine = dataCombine.set_index('기준일ID')
dataCombine

,기준일ID,행정동코드,시간대구분,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,여자20세부터24세생활인구수,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수
0,20200601,11110,0,612,502,396,566,421,419,451,617
761,20200601,11110,1,622,494,390,574,427,426,439,604
1097,20200601,11110,2,617,509,382,577,427,428,435,604
1272,20200601,11110,3,621,512,386,575,437,429,433,609
1769,20200601,11110,4,621,533,383,581,436,436,432,617
...,...,...,...,...,...,...,...,...,...,...,...
934129,20200831,11740,19,630,809,858,914,778,906,897,1150
934576,20200831,11740,20,656,845,922,990,793,912,961,1124
935242,20200831,11740,21,709,922,968,1033,808,962,943,1134
935433,20200831,11740,22,741,934,1023,1055,848,990,954,1171


In [74]:
code = pd.read_excel('행정동_20200814.xlsx')
seoul_code = code.loc[code['시도명'] == '서울특별시']
seoul_code = seoul_code.drop(['시도명','읍면동명'], axis=1)
seoul_code = seoul_code.dropna()
# slicing을 위해 행정동코드를 문자열로 바꾸고 10자리인 행정동코드를 slicing으로 8자리로 변경
seoul_code = seoul_code.astype(str)
seoul_code['행정동코드'] = seoul_code['행정동코드'].str[0:5]
seoul_code = seoul_code.drop_duplicates()
seoul_code

,행정동코드,시군구명
1,11110,종로구
19,11140,중구
35,11170,용산구
52,11200,성동구
70,11215,광진구
86,11230,동대문구
101,11260,중랑구
118,11290,성북구
139,11305,강북구
153,11320,도봉구


In [83]:
local_people = pd.merge(dataCombine, seoul_code, how='right', on='행정동코드')
local_people = local_people[['기준일ID','시군구명','시간대구분',
                             '남자20세부터24세생활인구수','남자25세부터29세생활인구수',
                             '남자30세부터34세생활인구수','남자35세부터39세생활인구수',
                             '여자20세부터24세생활인구수','여자25세부터29세생활인구수',
                             '여자30세부터34세생활인구수','여자35세부터39세생활인구수']]

local_people = local_people.rename(columns={'시군구명':'군구명',
                                            '시간대구분':'시간',
                                            '남자20세부터24세생활인구수':'남20~24세',
                                            '남자25세부터29세생활인구수':'남25~29세',
                                            '남자30세부터34세생활인구수':'남30~34세',
                                            '남자35세부터39세생활인구수':'남35~39세',
                                            '여자20세부터24세생활인구수':'여20~24세',
                                            '여자25세부터29세생활인구수':'여25~29세',
                                            '여자30세부터34세생활인구수':'여30~34세',
                                            '여자35세부터39세생활인구수':'여35~39세'})
local_people = local_people.groupby(['기준일ID','군구명','시간']).mean()
local_people = local_people.round(0)
local_people = local_people.astype(int)
# 맨 위에 save_file 변수에 지정한 파일 경로와 파일명으로 저장
local_people.to_csv(save_file, encoding = 'utf-8-sig')
local_people

남20~24세  남25~29세  남30~34세  남35~39세  여20~24세  여25~29세  \
기준일ID    군구명 시간                                                         
20200601 강남구 0       790     1064     1108     1311     1013     1188   
             1       787     1048     1105     1303     1004     1182   
             2       781     1047     1093     1295     1001     1179   
             3       779     1043     1087     1291      999     1179   
             4       776     1039     1089     1293     1001     1180   
...                  ...      ...      ...      ...      ...      ...   
20200831 중랑구 19      619      764      733      882      750      874   
             20      632      799      765      927      775      914   
             21      636      830      796      969      803      948   
             22      644      843      812      993      814      963   
             23      655      862      834     1017      818      983   

                 여30~34세  여35~39세  
기준일ID    군구명 시간                    
20200601 강남구 0      1245     1399  
             1      1240     1400  
             2      1239     1397  
             3      1238     1398  
             4      1238     1398  
...                  ...      ...  
20200831 중랑구 19      835      928  
             20      873      952  
             21      894      969  
             22      911      986  
             23      928     1001  

[55200 rows x 8 columns]